In [1]:
import hashlib
import math
from typing import List, Tuple

# ======== SM3 哈希（此处使用 sha256 模拟） =========
def sm3_hash(data: bytes) -> bytes:
    return hashlib.sha256(data).digest()  # 替换为真实 SM3 哈希

# ======== RFC6962 Merkle Tree 实现 ==========
class MerkleTreeRFC6962:
    def __init__(self, leaves: List[bytes]):
        self.leaves = [sm3_hash(b'\x00' + leaf) for leaf in leaves]  # Leaf prefix 0x00
        self.levels = []
        self.build_tree()

    def build_tree(self):
        nodes = self.leaves[:]
        self.levels.append(nodes)
        while len(nodes) > 1:
            next_level = []
            for i in range(0, len(nodes), 2):
                left = nodes[i]
                if i + 1 < len(nodes):
                    right = nodes[i + 1]
                else:
                    right = left
                parent = sm3_hash(b'\x01' + left + right)  # Internal prefix 0x01
                next_level.append(parent)
            nodes = next_level
            self.levels.append(nodes)
        self.root = self.levels[-1][0] if self.levels else None

    def get_root(self) -> bytes:
        return self.root

    def get_proof(self, index: int) -> List[Tuple[bytes, bool]]:
        proof = []
        i = index
        for level in self.levels[:-1]:
            if i % 2 == 0:
                if i + 1 < len(level):
                    proof.append((level[i + 1], True))  # 右兄弟
            else:
                proof.append((level[i - 1], False))     # 左兄弟
            i = i // 2
        return proof

    @staticmethod
    def verify_proof(leaf: bytes, index: int, proof: List[Tuple[bytes, bool]], root: bytes) -> bool:
        computed_hash = sm3_hash(b'\x00' + leaf)
        for sibling_hash, is_right in proof:
            if is_right:
                computed_hash = sm3_hash(b'\x01' + computed_hash + sibling_hash)
            else:
                computed_hash = sm3_hash(b'\x01' + sibling_hash + computed_hash)
        return computed_hash == root

    def get_leaf_index(self, leaf_data: bytes) -> int:
        leaf_hash = sm3_hash(b'\x00' + leaf_data)
        try:
            return self.leaves.index(leaf_hash)
        except ValueError:
            return -1

# ========= 构建 100000 个叶子节点 =========
leaf_count = 100_000
leaves = [f"leaf-{i}".encode() for i in range(leaf_count)]
tree = MerkleTreeRFC6962(leaves)
root = tree.get_root()
print("🌳 Merkle Root:", root.hex())

# ========= 存在性证明示例 =========
target_index = 12345
target_leaf = leaves[target_index]
proof = tree.get_proof(target_index)
valid = MerkleTreeRFC6962.verify_proof(target_leaf, target_index, proof, root)
print(f"✅ Inclusion Proof for leaf-{target_index}: {valid}")

# ========= 不存在性证明思路（逻辑演示） =========
nonexistent_leaf = b"leaf-100001"  # 不存在的节点
index = tree.get_leaf_index(nonexistent_leaf)
if index == -1:
    print("❌ Leaf not in tree, non-inclusion confirmed.")
else:
    print("⚠️ Leaf found, inclusion proof available.")


🌳 Merkle Root: d8a6d0dfdc6a356d2705a499e9699dbd4a3ebfc2dc89419f80553a9c314c4afd
✅ Inclusion Proof for leaf-12345: True
❌ Leaf not in tree, non-inclusion confirmed.
